In [3]:
# !pip install selenium

from selenium import webdriver      # 웹 접근 모듈
from bs4 import BeautifulSoup as bs # 분석 용이 정제 모듈
import pandas as pd                 # 데이터 분석 모듈 

In [72]:
driver = webdriver.Chrome('chromedriver.exe') 
# 크롬 브라우저를 파이썬으로 컨트롤하기 위한 모듈 파일 임포트
# 사용중인 브라우저의 버전을 꼭 확인하여 해당 버전에 맞는
# chromedriver.exe 다운로드

driver.get('https://www.starbucks.co.kr/store/store_map.do')
# 분석 할 대상 주소로 이동

C:\Users\KOREA_IT\AppData\Local\Temp/ipykernel_13904/2115985747.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [10]:
area = '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'
driver.find_element_by_css_selector(area).click()

# 클릭할 단추를 우클릭 -> 검사 -> 검사 ->그 위치를 우클릭 -> copy -> copy selector

C:\Users\KOREA_IT\AppData\Local\Temp/ipykernel_13904/3961443305.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(area).click()


In [12]:
seoul = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(seoul).click()

C:\Users\KOREA_IT\AppData\Local\Temp/ipykernel_13904/3017916038.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(seoul).click()


In [13]:
all = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(all).click()


C:\Users\KOREA_IT\AppData\Local\Temp/ipykernel_13904/943879005.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(all).click()


In [14]:
html = driver.page_source
soup = bs(html)

In [17]:
len(soup.select('li'))

1029

In [18]:
len(soup.select('li.quickResultLstCon'))

587

In [20]:
stars = soup.select('li.quickResultLstCon')
star = stars[0]
star

<li class="quickResultLstCon" data-code="9626" data-hlytag="null" data-index="0" data-lat="37.5115577" data-long="127.03226199999994" data-name="논현힐탑" data-storecd="430" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="논현힐탑" data-store="430" data-yn="N">논현힐탑  </strong> <p class="result_details">서울특별시 강남구 논현로 648 (논현동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [22]:
# 위도와 경도 추출 data-lat   data-long
# 키값과 밸류값 형태의 경우에는 딕셔너리 형태로 추출 할수 있다. 
lat = star['data-lat']
lng = star['data-long']
lng

'127.03226199999994'

In [23]:
name = star['data-name']
name

'논현힐탑'

In [24]:
star.select('p')

[<p class="result_details">서울특별시 강남구 논현로 648 (논현동)<br/>1522-3232</p>]

In [25]:
star.select('p')[0].text

# 단순히 글자들을 추출하게 되면 전화 번호의 경우 지역별 
# 또는 국번의 개수가 전체 지점이 일치하지 않을 수도 있다. 
# 따라서 인덱싱이나 슬라이싱으로 분리하기는 적당하지 않다. 

'서울특별시 강남구 논현로 648 (논현동)1522-3232'

In [26]:
star.select('p')

# 모든 지점의 주소와 전화번호는 항상 <br/> 로 분리 되어 있다고 한다. 
# 따라서 <br/> 를 기준으로 앞부분은 주소로 뒷부분은 전번으로 추출하자


[<p class="result_details">서울특별시 강남구 논현로 648 (논현동)<br/>1522-3232</p>]

In [28]:
# star.select('p').split('<br/>')[0]
# 에러 발생 !!! split은 글자를 분리할수 있다 객체는 분라할수 없다. 

In [29]:
str(star.select('p')).split('<br/>')[0]

'[<p class="result_details">서울특별시 강남구 논현로 648 (논현동)'

In [30]:
str(star.select('p')).split('<br/>')[0].split('>')[1]

'서울특별시 강남구 논현로 648 (논현동)'

In [31]:
str(star.select('p')).split('<br/>')[1]

'1522-3232</p>]'

In [32]:
str(star.select('p')).split('<br/>')[1].split('<')[0]

'1522-3232'

In [35]:
starbucks = []

stars = soup.select('li.quickResultLstCon')

for star in stars:
    name = star['data-name']
    lat = star['data-lat']
    lng = star['data-long'] 
    addr = str(star.select('p')).split('<br/>')[0].split('>')[1]
    tel = str(star.select('p')).split('<br/>')[1].split('<')[0]

    starbucks.append([name, lat, lng, addr, tel])

df = pd.DataFrame(starbucks, columns = ['매장명','위도','경도','주소','전화'])
df.to_excel('starbuckslist.xlsx', index=False)

In [42]:
dong_names = []

for addr in df['주소']:
    dong = addr.split()[1]
    dong_names.append(dong)

df['시군구명'] = dong_names


In [44]:
df.pivot_table(index='시군구명',
values='매장명',
aggfunc='count').rename(columns={'매장명':'매장수'})

,매장수
시군구명,
강남구,94
강동구,16
강북구,6
강서구,23
관악구,12
광진구,20
구로구,12
금천구,12
노원구,13


In [47]:
#!pip install folium

import folium    # 파이썬에서 지도 그리기 모듈

In [49]:
# 기본 지도 그리기

map = folium.Map()   # 옵션없이 기본 지도는 전세계
map

In [52]:
map = folium.Map(
        location = [37.566640, 126.977458]   # 지도의 기준 잡기
)

map

In [62]:
map = folium.Map(
        location = [37.566640, 126.977458],   # 지도의 기준 잡기
        zoom_start = 11                       # 지도 확대
)

map

In [63]:
map = folium.Map(
        location = [37.566640, 126.977458],   # 지도의 기준 잡기
        zoom_start = 11,                      # 지도 확대
        tiles = 'Stamen Terrain'
)

map

In [77]:
map = folium.Map(
        location = [37.566640, 126.977458],   # 지도의 기준 잡기
        zoom_start = 11,                      # 지도 확대
        tiles = 'Stamen Toner'
)

map

In [81]:
map = folium.Map(
        location = [37.566640, 126.977458],   # 지도의 기준 잡기
        zoom_start = 11,                      # 지도 확대
        tiles = 'CartoDB Positron'
)

map

In [70]:
for idx in df.index:
    lat = df.loc[idx,'위도']
    lng = df.loc[idx,'경도']
    
    folium.Marker(
        location=[lat, lng]
    ).add_to(map)

map

In [71]:
# 587  -> 577  (집 주변 매장 10개를 한번 더 카운트 )

df.drop_duplicates(['매장명'])

,매장명,위도,경도,주소,전화,시군구명
0,논현힐탑,37.5115577,127.03226199999994,서울특별시 강남구 논현로 648 (논현동),1522-3232,강남구
1,학동역,37.51464884,127.0307302,서울특별시 강남구 논현로 704 (논현동),1522-3232,강남구
2,강남논현,37.5078978596254,127.023338614644,서울특별시 강남구 강남대로 512 (논현동),1522-3232,강남구
3,차병원사거리,37.508032,127.035373,"서울특별시 강남구 봉은사로 213, 센트럴타워 (논현동)",1522-3232,강남구
4,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구
...,...,...,...,...,...,...
582,중랑구청,37.60538908,127.0957558,서울특별시 중랑구 신내로 72,1522-3232,중랑구
583,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232,중랑구
584,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,중랑구
585,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,중랑구


In [ ]:
# 제주도의 스타벅스 매장 데이터를 클롤링 하여 folium을 사용하여 지도를 그려 제출하세요


In [82]:
for idx in df.index:
    lat = df.loc[idx,'위도']
    lng = df.loc[idx,'경도']
    name = df.loc[idx,'매장명']
    addr = df.loc[idx,'주소']
    
    popup =  folium.Popup("매장명 : " + "<h5>" + name + "</h5><br>" + "주소 : " + addr, Min_width=200, max_width=200)

    folium.Marker(
        location=[lat, lng],
        popup = popup,
    ).add_to(map)

map

In [83]:
driver = webdriver.Chrome('chromedriver.exe') 

driver.get('https://www.starbucks.co.kr/store/store_map.do')

C:\Users\KOREA_IT\AppData\Local\Temp/ipykernel_13904/929490355.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [84]:
area = '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'
driver.find_element_by_css_selector(area).click()

C:\Users\KOREA_IT\AppData\Local\Temp/ipykernel_13904/3427700919.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(area).click()


In [85]:
jeju = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(16) > a'
driver.find_element_by_css_selector(jeju).click()

C:\Users\KOREA_IT\AppData\Local\Temp/ipykernel_13904/3024342062.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(jeju).click()


In [86]:

all = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element_by_css_selector(all).click()

C:\Users\KOREA_IT\AppData\Local\Temp/ipykernel_13904/2471459405.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(all).click()


In [87]:
html = driver.page_source
soup = bs(html)

In [88]:
starbucksjeju = []

stars = soup.select('li.quickResultLstCon')

for star in stars:
    name = star['data-name']
    lat = star['data-lat']
    lng = star['data-long'] 
    addr = str(star.select('p')).split('<br/>')[0].split('>')[1]
    tel = str(star.select('p')).split('<br/>')[1].split('<')[0]

    starbucksjeju.append([name, lat, lng, addr, tel])

dfjeju = pd.DataFrame(starbucksjeju, columns = ['매장명','위도','경도','주소','전화'])
dfjeju.to_excel('starbuckslistjeju.xlsx', index=False)

In [89]:
dfjeju

,매장명,위도,경도,주소,전화
0,논현힐탑,37.5115577,127.03226199999994,서울특별시 강남구 논현로 648 (논현동),1522-3232
1,학동역,37.51464884,127.0307302,서울특별시 강남구 논현로 704 (논현동),1522-3232
2,강남논현,37.5078978596254,127.023338614644,서울특별시 강남구 강남대로 512 (논현동),1522-3232
3,차병원사거리,37.508032,127.035373,"서울특별시 강남구 봉은사로 213, 센트럴타워 (논현동)",1522-3232
4,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232
5,강남대로논현R,37.507413,127.022568,서울특별시 서초구 강남대로85길 5 (반포동),1522-3232
6,서울세관사거리,37.514887,127.035653,서울특별시 강남구 언주로 650 (논현동) 한국건설기술인협회,1522-3232
7,논현역,37.5111983,127.0205875,서울특별시 서초구 강남대로 557 (잠원동) 1~2층,1522-3232
8,신논현역,37.50529,127.023636,서울특별시 서초구 강남대로 483 (반포동) 청호빌딩,1522-3232
9,신사역성일빌딩,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동),1522-3232


In [93]:
mapjeju = folium.Map(
        location = [33.5073284, 126.5408980],   # 지도의 기준 잡기
        zoom_start = 10,                        # 지도 확대
        tiles = 'CartoDB Positron'
)

mapjeju

In [119]:
for idx in dfjeju.index:
    lat = dfjeju.loc[idx,'위도']
    lng = dfjeju.loc[idx,'경도']
    name = dfjeju.loc[idx,'매장명']
    addr = dfjeju.loc[idx,'주소']
    tel = dfjeju.loc[idx,'전화']
    
    popup =  folium.Popup("매장명 : " + "<h4>&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp&nbsp<font color='Red'>" + name + "</font></h4><br>" + "주소 : <p><font size=1><b>" + addr + "</b></font></p>전화 : " + tel, Min_width=200, max_width=200)

    folium.Marker(
        location=[lat, lng],
        popup = popup,
    ).add_to(mapjeju)

mapjeju

In [98]:
dfjeju = dfjeju[~dfjeju['주소'].str.contains('서울특별시')]
dfjeju

,매장명,위도,경도,주소,전화
10,제주신화월드R,33.306608,126.316714,제주특별자치도 서귀포시 안덕면 신화역사로304번길 38 B1-29,1522-3232
11,제주에듀시티,33.291788,126.284407,제주특별자치도 서귀포시 대정읍 에듀시티로 36,1522-3232
12,제주성산DT,33.449980,126.920605,제주특별자치도 서귀포시 성산읍 일출로 80,1522-3232
13,제주중문DT,33.2549983,126.41590589999998,제주특별자치도 서귀포시 천제연로 95(색달동),1522-3232
14,제주송악산,33.206799,126.290035,제주특별자치도 서귀포시 대정읍 형제해안로 322,1522-3232
15,서귀포DT,33.2492775,126.50652289999993,제주특별자치도 서귀포시 일주서로 11 (강정동),1522-3232
16,성산일출봉,33.462865,126.935544,"제주특별자치도 서귀포시 성산읍 일출로 284-5 1,2층",1522-3232
17,제주중문,33.251128,126.412752,제주특별자치도 서귀포시 중문관광로110번길 32 (색달동),1522-3232
18,서귀포올레,33.2483024645772,126.563777617613,제주특별자치도 서귀포시 중정로 69 (서귀동),1522-3232
19,제주도남DT,33.480565,126.523646,제주특별자치도 제주시 연북로 394 (도남동),1522-3232


In [99]:
dfjeju.reset_index(drop=True, inplace=True)
dfjeju

,매장명,위도,경도,주소,전화
0,제주신화월드R,33.306608,126.316714,제주특별자치도 서귀포시 안덕면 신화역사로304번길 38 B1-29,1522-3232
1,제주에듀시티,33.291788,126.284407,제주특별자치도 서귀포시 대정읍 에듀시티로 36,1522-3232
2,제주성산DT,33.449980,126.920605,제주특별자치도 서귀포시 성산읍 일출로 80,1522-3232
3,제주중문DT,33.2549983,126.41590589999998,제주특별자치도 서귀포시 천제연로 95(색달동),1522-3232
4,제주송악산,33.206799,126.290035,제주특별자치도 서귀포시 대정읍 형제해안로 322,1522-3232
5,서귀포DT,33.2492775,126.50652289999993,제주특별자치도 서귀포시 일주서로 11 (강정동),1522-3232
6,성산일출봉,33.462865,126.935544,"제주특별자치도 서귀포시 성산읍 일출로 284-5 1,2층",1522-3232
7,제주중문,33.251128,126.412752,제주특별자치도 서귀포시 중문관광로110번길 32 (색달동),1522-3232
8,서귀포올레,33.2483024645772,126.563777617613,제주특별자치도 서귀포시 중정로 69 (서귀동),1522-3232
9,제주도남DT,33.480565,126.523646,제주특별자치도 제주시 연북로 394 (도남동),1522-3232
